# RAG Tools: Database Setup

## 1. Introduction

In this notebook, we'll set up Docker containers for our PostgreSQL database with pgvector extension and Neo4j graph database. We'll ensure both databases have accessible browser interfaces for easy management and are isolated in their own Docker network.

## 2. Databases - PgVector and Neo4j

"We're using a combination of PgVector (PostgreSQL with vector extensions) and Neo4j to create a powerful RAG (Retrieval-Augmented Generation) system. PgVector allows us to store and efficiently query high-dimensional vectors, which are crucial for embedding-based search and similarity comparisons in machine learning applications.
Neo4j, being a graph database, excels at representing and querying complex relationships between entities. By combining these two databases, we can create a system that not only understands semantic similarity (through vector embeddings) but also captures and leverages the intricate relationships between different pieces of information.
This combination is particularly powerful for tasks like code analysis, where we need to understand both the content of code (which can be embedded into vectors) and the relationships between different code elements (which can be represented as a graph).

## 3. Environment Configuration

First, let's set up our environment variables. We'll create a file named `.env` in the `config/` directory:

The .env file plays a crucial role in our framework as a centralized repository for environment-specific configuration variables. As we progress through our project, we'll continually add new variables to this file, allowing us to easily manage and update our configuration settings without modifying our code. The .env file stores sensitive information like database credentials, API keys, and other configuration parameters that may vary between development, testing, and production environments. To leverage these variables effectively, we've created the config_utils tool, which acts as an interface between our application and the .env file. The Config class within config_utils uses the python-dotenv library to load variables from the .env file, making them accessible throughout our application. This approach offers several advantages: it enhances security by keeping sensitive information out of our codebase, promotes flexibility by allowing easy configuration changes without code modifications, and improves maintainability by centralizing our configuration management. As we add new components or services to our framework, we'll update both the .env file and the Config class in config_utils, ensuring that our entire application remains in sync with our latest configuration needs.

We have provided an example.env file located in config/ simply open it up in a text editor of your choice, update the passwords and save it as .env.


## 4. Docker Compose Configuration

Let's examine and update our `docker-compose.yml` file. This configuration sets up our PostgreSQL and Neo4j databases in isolated containers with persistent storage.

```yaml
version: '3.8'

services:
  postgres:
    container_name: ${POSTGRES_CONTAINER_NAME}
    environment:
      POSTGRES_DB: ${POSTGRES_DB}
      POSTGRES_PASSWORD: ${POSTGRES_PASSWORD}
      POSTGRES_USER: ${POSTGRES_USER}
    image: ankane/pgvector
    networks:
    - ragtools_network
    ports:
    - ${POSTGRES_PORT}:5432
    volumes:
    - ../db_data/postgres:/var/lib/postgresql/data

  neo4j:
    image: neo4j:latest
    container_name: ${NEO4J_CONTAINER_NAME}
    environment:
      NEO4J_AUTH: ${NEO4J_AUTH}
    ports:
      - "${NEO4J_HTTP_PORT}:7474"
      - "${NEO4J_BOLT_PORT}:7687"
    volumes:
      - ../db_data/neo4j:/data
    networks:
      - ragtools_network

networks:
  ragtools_network:
    name: ${DOCKER_NETWORK_NAME}

```

Let's break down the key components of this configuration:

1. **Image Selection**: 
   - For PostgreSQL, we use the `ankane/pgvector` image, which comes with the pgvector extension pre-installed.
   - For Neo4j, we use the latest official Neo4j image.

2. **Environment Variables**: 
   - We use environment variables (${VAR_NAME}) to configure the containers. These variables are defined in our `.env` file.

3. **Port Mapping**: 
   - We map the container ports to host ports, allowing access from the host machine.

4. **Volumes**: 
   - For both PostgreSQL and Neo4j, we've updated the volume mappings to use the `db_data` directory:
     - `../db_data/postgres:/var/lib/postgresql/data` for PostgreSQL
     - `../db_data/neo4j:/data` for Neo4j
   - This ensures that our data persists even if the containers are stopped or removed.

5. **Networking**: 
   - Both containers are connected to a custom network named `ragtools_network`, isolating them from other Docker networks on the system.

Now, let's create this `docker-compose.yml` file in our `config` directory:

In [ ]:
import os
import yaml

# Get the current working directory (assuming we're in the notebook directory)
current_dir = os.getcwd()

# Construct the path to the config directory (two levels up from the notebook)
config_dir = os.path.join(current_dir, '..', '..', 'config')

# Ensure the config directory exists
os.makedirs(config_dir, exist_ok=True)

# Construct the full path for the docker-compose.yml file
docker_compose_path = os.path.join(config_dir, 'docker-compose.yml')

# Docker Compose configuration
docker_compose_config = {
    'version': '3.8',
    'services': {
        'postgres': {
            'image': 'ankane/pgvector',
            'container_name': '${POSTGRES_CONTAINER_NAME}',
            'environment': {
                'POSTGRES_DB': '${POSTGRES_DB}',
                'POSTGRES_USER': '${POSTGRES_USER}',
                'POSTGRES_PASSWORD': '${POSTGRES_PASSWORD}'
            },
            'ports': ['${POSTGRES_PORT}:5432'],
            'volumes': ['../db_data/postgres:/var/lib/postgresql/data'],
            'networks': ['ragtools_network']
        },
        'neo4j': {
            'image': 'neo4j:latest',
            'container_name': '${NEO4J_CONTAINER_NAME}',
            'environment': {
                'NEO4J_AUTH': '${NEO4J_AUTH}'
            },
            'ports': [
                '${NEO4J_HTTP_PORT}:7474',
                '${NEO4J_BOLT_PORT}:7687'
            ],
            'volumes': ['../db_data/neo4j:/data'],
            'networks': ['ragtools_network']
        }
    },
    'networks': {
        'ragtools_network': {
            'name': '${DOCKER_NETWORK_NAME}'
        }
    }
}

# Write the Docker Compose configuration to the file
with open(docker_compose_path, 'w') as f:
    yaml.dump(docker_compose_config, f, default_flow_style=False)

print(f"docker-compose.yml file created at: {docker_compose_path}")


This setup ensures that our PostgreSQL and Neo4j databases will store their data in the `db_data` directory, providing persistence across container restarts or removals. The `../db_data` path in the volume mappings is relative to the location of the `docker-compose.yml` file in the `config` directory, correctly pointing to the `db_data` directory at the project root.


## 5. Configuration Utility

To manage our configuration variables more efficiently, we'll create a `Config` class in a file named `config_utils.py`. This class will load environment variables from our `.env` file and provide easy access to these configurations throughout our project.

Here's the content for the `config_utils.py` file:

```python
import os
from dotenv import load_dotenv

class Config:
    def __init__(self):
        load_dotenv()
        
        print("Debug: Environment variables loaded")
        for key, value in os.environ.items():
            if key.startswith(('POSTGRES_', 'NEO4J_', 'DOCKER_')):
                print(f"Debug: {key} = {value}")

        # PostgreSQL configurations
        self.POSTGRES_DB = os.getenv('POSTGRES_DB')
        self.POSTGRES_USER = os.getenv('POSTGRES_USER')
        self.POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')
        self.POSTGRES_HOST = os.getenv('POSTGRES_HOST')
        self.POSTGRES_PORT = os.getenv('POSTGRES_PORT')
        
        # Neo4j configurations
        self.NEO4J_USER = os.getenv('NEO4J_USER', 'neo4j')
        self.NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
        self.NEO4J_AUTH = os.getenv('NEO4J_AUTH')
        if not self.NEO4J_AUTH:
            self.NEO4J_AUTH = f"{self.NEO4J_USER}/{self.NEO4J_PASSWORD}"
        self.NEO4J_HOST = os.getenv('NEO4J_HOST', 'localhost')
        self.NEO4J_HTTP_PORT = os.getenv('NEO4J_HTTP_PORT', '7474')
        self.NEO4J_BOLT_PORT = os.getenv('NEO4J_BOLT_PORT', '7687')
        
        # Docker configurations
        self.POSTGRES_CONTAINER_NAME = os.getenv('POSTGRES_CONTAINER_NAME')
        self.NEO4J_CONTAINER_NAME = os.getenv('NEO4J_CONTAINER_NAME')
        self.DOCKER_NETWORK_NAME = os.getenv('DOCKER_NETWORK_NAME')

        print("Debug: Config object initialized")
        self.print_all_attributes()

    def get_neo4j_connection_params(self):
        user, password = self.NEO4J_AUTH.split('/')
        return {
            "uri": f"bolt://{self.NEO4J_HOST}:{self.NEO4J_BOLT_PORT}",
            "auth": (user, password)
        }

    def get_postgres_connection_params(self):
        return {
            "dbname": self.POSTGRES_DB,
            "user": self.POSTGRES_USER,
            "password": self.POSTGRES_PASSWORD,
            "host": self.POSTGRES_HOST,
            "port": self.POSTGRES_PORT
        }

    def print_all_attributes(self):
        print("All Config attributes:")
        for attr, value in self.__dict__.items():
            print(f"{attr}: {value}")

    @classmethod
    def from_env(cls, env_path):
        # Load environment variables from a specific .env file
        load_dotenv(dotenv_path=env_path)
        return cls()

    def update_from_dict(self, config_dict):
        # Update configuration from a dictionary
        for key, value in config_dict.items():
            if hasattr(self, key):
                setattr(self, key, value)

    def to_dict(self):
        # Convert configuration to a dictionary
        return {k: v for k, v in self.__dict__.items() if not k.startswith('_')}
```

now lets put that file in place with the next code block:

In [ ]:
import os

# Get the current working directory
current_dir = os.getcwd()

# Construct the path to the src/utils directory
utils_dir = os.path.join(current_dir, '..', '..', 'src', 'utils')

# Ensure the utils directory exists
os.makedirs(utils_dir, exist_ok=True)

# Construct the full path for the config_utils.py file
config_utils_path = os.path.join(utils_dir, 'config_utils.py')

# Updated content of the config_utils.py file
config_utils_content = '''
import os
from dotenv import load_dotenv

import os
from dotenv import load_dotenv

class Config:
    def __init__(self):
        load_dotenv()
        
        print("Debug: Environment variables loaded")
        for key, value in os.environ.items():
            if key.startswith(('POSTGRES_', 'NEO4J_', 'DOCKER_')):
                print(f"Debug: {key} = {value}")

        # PostgreSQL configurations
        self.POSTGRES_DB = os.getenv('POSTGRES_DB')
        self.POSTGRES_USER = os.getenv('POSTGRES_USER')
        self.POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')
        self.POSTGRES_HOST = os.getenv('POSTGRES_HOST')
        self.POSTGRES_PORT = os.getenv('POSTGRES_PORT')
        
        # Neo4j configurations
        self.NEO4J_USER = os.getenv('NEO4J_USER', 'neo4j')
        self.NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
        self.NEO4J_AUTH = os.getenv('NEO4J_AUTH')
        if not self.NEO4J_AUTH:
            self.NEO4J_AUTH = f"{self.NEO4J_USER}/{self.NEO4J_PASSWORD}"
        self.NEO4J_HOST = os.getenv('NEO4J_HOST', 'localhost')
        self.NEO4J_HTTP_PORT = os.getenv('NEO4J_HTTP_PORT', '7474')
        self.NEO4J_BOLT_PORT = os.getenv('NEO4J_BOLT_PORT', '7687')
        
        # Docker configurations
        self.POSTGRES_CONTAINER_NAME = os.getenv('POSTGRES_CONTAINER_NAME')
        self.NEO4J_CONTAINER_NAME = os.getenv('NEO4J_CONTAINER_NAME')
        self.DOCKER_NETWORK_NAME = os.getenv('DOCKER_NETWORK_NAME')

        print("Debug: Config object initialized")
        self.print_all_attributes()

    def get_neo4j_connection_params(self):
        user, password = self.NEO4J_AUTH.split('/')
        return {
            "uri": f"bolt://{self.NEO4J_HOST}:{self.NEO4J_BOLT_PORT}",
            "auth": (user, password)
        }

    def get_postgres_connection_params(self):
        return {
            "dbname": self.POSTGRES_DB,
            "user": self.POSTGRES_USER,
            "password": self.POSTGRES_PASSWORD,
            "host": self.POSTGRES_HOST,
            "port": self.POSTGRES_PORT
        }

    def print_all_attributes(self):
        print("All Config attributes:")
        for attr, value in self.__dict__.items():
            print(f"{attr}: {value}")

    @classmethod
    def from_env(cls, env_path):
        # Load environment variables from a specific .env file
        load_dotenv(dotenv_path=env_path)
        return cls()

    def update_from_dict(self, config_dict):
        # Update configuration from a dictionary
        for key, value in config_dict.items():
            if hasattr(self, key):
                setattr(self, key, value)

    def to_dict(self):
        # Convert configuration to a dictionary
        return {k: v for k, v in self.__dict__.items() if not k.startswith('_')}
'''

# Write the content to the config_utils.py file
with open(config_utils_path, 'w') as f:
    f.write(config_utils_content)

print(f"Updated config_utils.py file created at: {config_utils_path}")


This `Config` class provides a centralized way to access our configuration variables. It loads the environment variables from the `.env` file and stores them as attributes. It also includes methods to easily retrieve connection parameters for our databases, update configurations, and convert to/from dictionary format for flexibility.

Key features of this Config class:

1. **Environment Variable Loading**: Uses `load_dotenv()` to load variables from the .env file.
2. **Flexible Configuration**: Includes methods to load from specific .env files, update from dictionaries, and convert to dictionaries.
3. **Connection Parameters**: Provides methods to get formatted connection parameters for both PostgreSQL and Neo4j.
4. **Debugging**: Includes a `print_all_attributes` method for easy configuration verification.

This design allows for easy extension as the project grows. New configuration parameters can be added to the `__init__` method, and new methods can be created for additional services or features.

To use this Config class in other parts of your project, you can import it like this:

```python
from src.utils.config_utils import Config

config = Config()
postgres_params = config.get_postgres_connection_params()
neo4j_params = config.get_neo4j_connection_params()
```

## 6. Create DockerComposeManager

In the future, we may want automation to spin up a Docker environment, so let's create a Python tool to help us manage that. DockerComposeManager will be that utility.

```python
import subprocess
import os
from dotenv import load_dotenv

class DockerComposeManager:
    def __init__(self, compose_file_path):
        self.compose_file_path = os.path.abspath(compose_file_path)
        load_dotenv(dotenv_path=os.path.join(os.path.dirname(self.compose_file_path), '.env'))

    def run_command(self, command):
        try:
            result = subprocess.run(
                f"docker compose -f {self.compose_file_path} {command}",
                shell=True, check=True, capture_output=True, text=True
            )
            print(result.stdout)
        except subprocess.CalledProcessError as e:
            print(f"Error executing command: {e}")
            print(e.stderr)

    def start_containers(self):
        self.run_command("up -d")

    def stop_containers(self):
        self.run_command("down")

    def show_container_status(self):
        self.run_command("ps")
```

Let's discuss this code and its significance in our project:

1. **Early Standardization**: By implementing this class now, we're setting a standard for how Docker operations will be handled throughout the project. This consistency will be invaluable as the project grows and more developers potentially join.

2. **Abstraction of Docker Commands**: The class abstracts Docker Compose commands into simple method calls. This abstraction makes it easier for team members who might not be Docker experts to manage containers.

3. **Error Handling and Logging**: The `run_command` method includes error handling, which will help catch and report issues early in the development process. This can be crucial for troubleshooting complex multi-container setups.

4. **Environment Variable Integration**: The class loads environment variables from the .env file, ensuring that our Docker operations are always using the correct configuration.

5. **Flexibility for Future Expansion**: While we currently only have methods for starting, stopping, and checking status, the structure allows easy addition of more complex operations in the future.

6. **Testing and Development Aid**: This class will be incredibly useful for setting up and tearing down test environments. Developers can easily spin up the entire stack or individual services as needed.

7. **Troubleshooting Tool**: The `show_container_status` method provides a quick way to check on the state of our Docker environment, which can be invaluable during development and debugging.

8. **Potential for CI/CD Integration**: As the project evolves, this class could be easily integrated into CI/CD pipelines for automated testing and deployment.

9. **Learning Opportunity**: This class serves as an excellent example of how to create utility classes that bridge application code with system operations, a valuable skill in DevOps and software engineering.

By implementing this DockerComposeManager now, we're not just preparing for future needs; we're establishing good practices from the start. It encourages thinking about operations and development as interconnected aspects of the project, which is a core principle in modern DevOps practices.

As we continue to develop PandorasLock, this class will likely evolve. We might add methods for scaling services, running database migrations, or performing health checks. By having this foundation in place early, we make it easier to implement these features consistently across the project.

Let's create this class in our project structure:

In [ ]:
import os

# Get the current working directory
current_dir = os.getcwd()

# Construct the path to the src/utils directory
utils_dir = os.path.join(current_dir, '..', 'src', 'utils')

# Ensure the utils directory exists
os.makedirs(utils_dir, exist_ok=True)

# Construct the full path for the DockerComposeManager.py file
docker_compose_manager_path = os.path.join(utils_dir, 'DockerComposeManager.py')

# Content of the DockerComposeManager.py file
docker_compose_manager_content = """
import subprocess
import os
from dotenv import load_dotenv

class DockerComposeManager:
    def __init__(self, compose_file_path):
        self.compose_file_path = os.path.abspath(compose_file_path)
        load_dotenv(dotenv_path=os.path.join(os.path.dirname(self.compose_file_path), '.env'))

    def run_command(self, command):
        try:
            result = subprocess.run(
                f"docker compose -f {self.compose_file_path} {command}",
                shell=True, check=True, capture_output=True, text=True
            )
            print(result.stdout)
        except subprocess.CalledProcessError as e:
            print(f"Error executing command: {e}")
            print(e.stderr)

    def start_containers(self):
        self.run_command("up -d")

    def stop_containers(self):
        self.run_command("down")

    def show_container_status(self):
        self.run_command("ps")
"""

# Write the content to the DockerComposeManager.py file
with open(docker_compose_manager_path, 'w') as f:
    f.write(docker_compose_manager_content)

print(f"DockerComposeManager.py file created at: {docker_compose_manager_path}")


## 7. Setup Directory Structure and Launch Docker Containers

In this final section, we'll set up our directory structure for persistent databases and then use our newly created tools to launch the Docker containers.

First, let's create the necessary directories:

In [ ]:
import os
import sys
import time
import psycopg2
from neo4j import GraphDatabase

# Get the project root directory and add it to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
sys.path.append(project_root)

from src.utils.DockerComposeManager import DockerComposeManager
from src.utils.config_utils import Config

def create_directories(config):
    # Create a directory for database persistence
    db_dir = os.path.join(project_root, 'db_data')
    os.makedirs(db_dir, exist_ok=True)

    # Create subdirectories for each database
    postgres_dir = os.path.join(db_dir, 'postgres')
    neo4j_dir = os.path.join(db_dir, 'neo4j')

    os.makedirs(postgres_dir, exist_ok=True)
    os.makedirs(neo4j_dir, exist_ok=True)

    print(f"Created database directories:")
    print(f"PostgreSQL: {postgres_dir}")
    print(f"Neo4j: {neo4j_dir}")

def verify_env_variables(config):
    print("\nVerifying environment variables:")
    print(f"POSTGRES_DB: {config.POSTGRES_DB}")
    print(f"POSTGRES_USER: {config.POSTGRES_USER}")
    print(f"POSTGRES_HOST: {config.POSTGRES_HOST}")
    print(f"POSTGRES_PORT: {config.POSTGRES_PORT}")
    print(f"NEO4J_HOST: {config.NEO4J_HOST}")
    print(f"NEO4J_HTTP_PORT: {config.NEO4J_HTTP_PORT}")
    print(f"NEO4J_BOLT_PORT: {config.NEO4J_BOLT_PORT}")
    print(f"DOCKER_NETWORK_NAME: {config.DOCKER_NETWORK_NAME}")

def wait_for_postgres(config, max_attempts=5, delay=5):
    for attempt in range(max_attempts):
        try:
            conn = psycopg2.connect(
                dbname=config.POSTGRES_DB,
                user=config.POSTGRES_USER,
                password=config.POSTGRES_PASSWORD,
                host=config.POSTGRES_HOST,
                port=config.POSTGRES_PORT
            )
            conn.close()
            print(f"Successfully connected to PostgreSQL on port {config.POSTGRES_PORT}")
            return True
        except psycopg2.OperationalError as e:
            print(f"Attempt {attempt + 1}/{max_attempts}: PostgreSQL is not ready yet. Error: {e}")
            time.sleep(delay)
    return False

def wait_for_neo4j(config, max_attempts=5, delay=5):
    for attempt in range(max_attempts):
        try:
            print(f"Debug: Attempting to connect to Neo4j (Attempt {attempt + 1})")
            print(f"Debug: NEO4J_AUTH = {config.NEO4J_AUTH}")
            print(f"Debug: NEO4J_HOST = {config.NEO4J_HOST}")
            print(f"Debug: NEO4J_BOLT_PORT = {config.NEO4J_BOLT_PORT}")
            
            neo4j_params = config.get_neo4j_connection_params()
            print(f"Debug: Neo4j connection params: {neo4j_params}")
            
            driver = GraphDatabase.driver(
                neo4j_params['uri'],
                auth=neo4j_params['auth']
            )
            with driver.session() as session:
                result = session.run("RETURN 1 AS x")
                assert result.single()['x'] == 1
            driver.close()
            print(f"Successfully connected to Neo4j on port {config.NEO4J_BOLT_PORT}")
            return True
        except Exception as e:
            print(f"Attempt {attempt + 1}/{max_attempts}: Neo4j is not ready yet. Error: {e}")
            print(f"Error type: {type(e)}")
            print(f"Error details: {str(e)}")
            time.sleep(delay)
    return False

def verify_database_connections(config):
    postgres_ready = wait_for_postgres(config)
    neo4j_ready = wait_for_neo4j(config)

    if postgres_ready and neo4j_ready:
        print("\nAll database connections are successful!")
    else:
        print("\nSome database connections failed. Please check your configuration and container logs.")

def main():
    # Load configuration
    config = Config()

    # Create necessary directories
    create_directories(config)

    # Verify environment variables
    verify_env_variables(config)

    # Create an instance of DockerComposeManager
    docker_compose_path = os.path.join(project_root, 'config', 'docker-compose.yml')
    docker_manager = DockerComposeManager(docker_compose_path)

    # Start the containers
    print("\nStarting Docker containers...")
    docker_manager.start_containers()

    # Wait for containers to fully start
    time.sleep(10)

    # Check the status of the containers
    print("\nChecking container status:")
    docker_manager.show_container_status()

    # Verify database connections
    verify_database_connections(config)

    # Print connection information
    print("\nConnection Information:")
    print(f"PostgreSQL: {config.POSTGRES_HOST}:{config.POSTGRES_PORT}")
    print(f"Neo4j (HTTP): {config.NEO4J_HOST}:{config.NEO4J_HTTP_PORT}")
    print(f"Neo4j (Bolt): {config.NEO4J_HOST}:{config.NEO4J_BOLT_PORT}")

if __name__ == "__main__":
    main()


## Conclusion

In this notebook, we've successfully set up a robust database environment for our RAG (Retrieval-Augmented Generation) system. Here's a summary of our key accomplishments:

1. **Database Infrastructure**: We've established a Docker-based setup with two powerful databases:
   - PostgreSQL with pgvector extension for efficient vector storage and similarity searches.
   - Neo4j for graph-based data representation and querying.

2. **Configuration Management**: We implemented a flexible `Config` class that:
   - Loads environment variables from a `.env` file.
   - Provides easy access to configuration settings throughout our project.
   - Handles both PostgreSQL and Neo4j configurations.

3. **Docker Environment**: We created a `docker-compose.yml` file that:
   - Sets up our PostgreSQL and Neo4j containers.
   - Uses environment variables for flexible configuration.
   - Establishes persistent storage for our databases.

4. **Automated Setup and Verification**: We developed a Python script that:
   - Creates necessary directories for database persistence.
   - Launches Docker containers using our DockerComposeManager.
   - Verifies successful connections to both PostgreSQL and Neo4j.

5. **Troubleshooting and Debugging**: Throughout this process, we encountered and resolved several challenges:
   - Corrected port conflicts for PostgreSQL.
   - Addressed issues with Neo4j authentication settings.
   - Implemented detailed error reporting and debugging output.

### Accessing the Neo4j Web Interface

As part of our setup, we've enabled access to Neo4j's powerful web interface, Neo4j Browser. Here's how you can connect to it:

1. Open your web browser and navigate to `http://localhost:7474`.
2. You'll be presented with a login screen.
3. Enter the following credentials:
   - Username: `neo4j` (or the username you set in your .env file)
   - Password: [The password you set in your .env file]
4. Once logged in, you'll have access to the Neo4j Browser interface where you can:
   - Run Cypher queries directly in the browser.
   - Visualize your graph data.
   - Explore Neo4j's built-in documentation and tutorials.

This interface is an invaluable tool for interacting with your graph database, allowing you to easily query, visualize, and manipulate your data as you develop your RAG system.

### Next Steps

In our next notebook, we will focus on integrating Ollama, a tool for running large language models, with our existing database setup. We'll cover:

1. Setting up an Ollama container and connecting it to our Docker network.
2. Establishing connections between Ollama and our PostgreSQL and Neo4j databases.
3. Implementing basic data flow between Ollama and our databases.
4. Testing the integrated system with simple RAG operations.

This upcoming integration will bring us closer to a fully functional RAG system, combining the power of large language models with our sophisticated database backend.

This database setup, including the accessible Neo4j web interface, forms the foundation of our RAG system, providing us with the capability to store, query, and visualize both vector embeddings and graph-structured data efficiently.

By methodically building our system component by component, we're creating a robust, flexible, and powerful tool for advanced natural language processing tasks. The challenges we've overcome in this notebook have not only resulted in a working database setup but have also deepened our understanding of configuration management, Docker containerization, and database connectivity in a complex system.